In [ ]:
%pip install feast

!feast version


In [ ]:
!feast version

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/data_source/diabetes.csv")


In [4]:
predictors_df = data.loc[:,data.columns!='Outcome']
target_df = data['Outcome']

In [5]:
timestamps = pd.date_range(end = pd.Timestamp.now(),
                           periods = len(data),freq = 'D').to_frame(name = 'event_timestamp', index = False)

In [6]:
predictors_df = pd.concat(objs = [predictors_df, timestamps], axis = 1)
target_df = pd.concat(objs = [target_df, timestamps], axis =1)

In [7]:
dataLen = len(data)
idsList = list(range(dataLen))
patient_ids = pd.DataFrame(data = idsList, columns = ['patient_id'])
predictors_df = pd.concat(objs = [predictors_df, patient_ids], axis = 1)
target_df = pd.concat(objs = [target_df, patient_ids], axis =1)

In [8]:
predictors_df.to_parquet(path='/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/data_source/predictor_df.parquet')
target_df.to_parquet(path='/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/data_source/target_df.parquet')


In [42]:
%cd /home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/src/mlProject/components/feature_repo/feature_repo
%pwd


[Errno 2] No such file or directory: '/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/src/mlProject/components/feature_repo/feature_repo'
/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/src/mlProject/feature_repo/feature_repo


'/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/src/mlProject/feature_repo/feature_repo'

In [ ]:
!feast init feature_repo``

In [43]:
!feast apply

Created entity patient
Created feature view predictors_df_feature_view
Created feature view target_df_feature_view

Created sqlite table feature_repo_predictors_df_feature_view
Created sqlite table feature_repo_target_df_feature_view



In [ ]:
%cd /home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/src/mlProject/feature_repo/feature_repo

### Generate Training Data    

In [44]:
from feast import FeatureStore
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

store = FeatureStore(repo_path = ".")
entity_df = pd.read_parquet('/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/data_source/target_df.parquet')
training_data = store.get_historical_features(
    entity_df = entity_df,
    features = [
        "predictors_df_feature_view:Pregnancies",
        "predictors_df_feature_view:Glucose",
        "predictors_df_feature_view:BloodPressure",
        "predictors_df_feature_view:SkinThickness",
        "predictors_df_feature_view:Insulin",
        "predictors_df_feature_view:BMI",
        "predictors_df_feature_view:DiabetesPedigreeFunction",
        "predictors_df_feature_view:Age",
               ]
)
dataset = store.create_saved_dataset(
    from_ = training_data,
    name = "diabetes_dataset",
    storage = SavedDatasetFileStorage('/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/src/mlProject/feature_repo/feature_repo/data/diabetes_dataset.parquet')
    )

/home/mpaul/projects/mpaul/mai/end-to-end-ml-pipeline-using-feast/.venv/lib/python3.8/site-packages/feast/feature_store.py:1177: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [46]:
# Importing dependencies
from feast import FeatureStore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from joblib import dump

# Getting our FeatureStore
store = FeatureStore(repo_path=".")

# Retrieving the saved dataset and converting it to a DataFrame
training_df = store.get_saved_dataset(name="diabetes_dataset").to_df()

# Separating the features and labels
y = training_df['Outcome']
X = training_df.drop(
    labels=['Outcome', 'event_timestamp', "patient_id"], 
    axis=1)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    stratify=y)

# Creating and training LogisticRegression
reg = LogisticRegression(max_iter = 200)
reg.fit(X=X_train[sorted(X_train)], y=y_train)

# Saving the model
dump(value=reg, filename="model.joblib")

['model.joblib']

In [47]:
# Importing dependencies
from feast import FeatureStore
from datetime import datetime, timedelta

# Getting our FeatureStore
store = FeatureStore(repo_path=".")

#store.materialize_incremental(end_date = datetime.now())

store.materialize(start_date=datetime.utcnow() - timedelta(days=530), end_date=datetime.utcnow() - timedelta(days=10))

Materializing 2 feature views from 2023-10-04 01:33:36-04:00 to 2025-03-07 01:33:36-05:00 into the sqlite online store.

predictors_df_feature_view:


100%|███████████████████████████████████████████████████████████| 520/520 [00:00<00:00, 2481.22it/s]


target_df_feature_view:


100%|██████████████████████████████████████████████████████████| 520/520 [00:00<00:00, 12454.96it/s]


In [50]:
# Importing dependencies
from feast import FeatureStore
import pandas as pd
from joblib import load

# Getting our FeatureStore
store = FeatureStore(repo_path=".")

# Defining our features names
feast_features = [
        "predictors_df_feature_view:Pregnancies",
        "predictors_df_feature_view:Glucose",
        "predictors_df_feature_view:BloodPressure",
        "predictors_df_feature_view:SkinThickness",
        "predictors_df_feature_view:Insulin",
        "predictors_df_feature_view:BMI",
        "predictors_df_feature_view:DiabetesPedigreeFunction",
        "predictors_df_feature_view:Age",
    ]

# Getting the latest features
features = store.get_online_features(
    features=feast_features,    
    entity_rows=[{"patient_id": 767}, {"patient_id": 766}]
).to_dict()

# Converting the features to a DataFrame
features_df = pd.DataFrame.from_dict(data=features)
print(features_df)

   patient_id Pregnancies BloodPressure Insulin   BMI  \
0         767        None          None    None  None   
1         766        None          None    None  None   

  DiabetesPedigreeFunction SkinThickness Glucose   Age  
0                     None          None    None  None  
1                     None          None    None  None  


In [49]:
# Loading our model and doing inference
reg = load("model.joblib")
predictions = reg.predict(features_df[sorted(features_df.drop("patient_id", axis=1))])
print(predictions)
prediction_probabilities = reg.predict_proba(features_df[sorted(features_df.drop("patient_id", axis=1))])
print(prediction_probabilities)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values